## קוד מבוא

In [137]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [138]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## פונקציות

In [139]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [140]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [141]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [142]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [143]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [144]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [145]:
def rename_text_in_col(df,file_path,file_name,old_v,new_v,col_name):
    
    col_convert=up_load_df(file_path,file_name).set_index(old_v)

    df=df.set_index(col_name)

    df['new_col']=col_convert[new_v]

    df=df.reset_index()

    df[col_name]=df['new_col']

    df=df.drop(columns='new_col')
    
    return df

In [146]:
def sum_df_pop(v4,v3,col_to_index,name):
    
    v4_pivot=v4.pivot_table(index=col_to_index,aggfunc=sum)

    v3_pivot=v3.pivot_table(index=col_to_index,aggfunc=sum)
    
    df_v4_v3=v4_pivot.merge(v3_pivot,left_index=True,right_index=True,how='outer',suffixes=('_v4','_v3')).fillna(0)

    df_v4_v3['aprt_kayim_delta_pre']=df_v4_v3['aprt_20_v4']/df_v4_v3['aprt_20_v3']

    df_v4_v3['pop_kayim_delta_pre']=df_v4_v3['pop_without_dorms_yeshiva_v4']/df_v4_v3['pop_without_dorms_yeshiva_v3']

    df_v4_v3['pop_2040_delta_pre']=df_v4_v3['pop_v4']/df_v4_v3['pop_v3']

    df_v4_v3['aprt_2040_delta_pre']=df_v4_v3['aprt_v4']/df_v4_v3['aprt_v3']

    df_v4_v3['add_aprt_2040_v4']=df_v4_v3['aprt_v4']-df_v4_v3['aprt_20_v4']

    df_v4_v3['add_aprt_2040_v3']=df_v4_v3['aprt_v3']-df_v4_v3['aprt_20_v3']

    df_v4_v3['add_pop_2040_v3']=df_v4_v3['pop_v3']-df_v4_v3['pop_without_dorms_yeshiva_v3']

    df_v4_v3['add_pop_2040_v4']=df_v4_v3['pop_v4']-df_v4_v3['pop_without_dorms_yeshiva_v4']

    df_v4_v3['add_pop_2040_delta_pre']=df_v4_v3['add_pop_2040_v4']/df_v4_v3['add_pop_2040_v3']

    df_v4_v3['add_aprt_2040_delta_pre']=df_v4_v3['add_aprt_2040_v4']/df_v4_v3['add_aprt_2040_v3']

    

    col=['aprt_20_v3', 'aprt_20_v4','aprt_kayim_delta_pre','pop_without_dorms_yeshiva_v3','pop_without_dorms_yeshiva_v4','pop_kayim_delta_pre']

    df_v4_v3[col].reset_index().to_excel('{}\{}_pop_kayim_comparison.xlsx'.format(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\output',name),index=False)

    col=['aprt_v3','aprt_v4','aprt_2040_delta_pre','pop_v3','pop_v4','pop_2040_delta_pre']

    df_v4_v3[col].reset_index().to_excel('{}\{}_pop_2040_comparison.xlsx'.format(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\output',name),index=False)

    col=['add_aprt_2040_v3','add_aprt_2040_v4','add_aprt_2040_delta_pre','add_pop_2040_v3','add_pop_2040_v4','add_pop_2040_delta_pre']

    df_v4_v3[col].reset_index().to_excel('{}\{}_pop_add_comparison.xlsx'.format(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\output',name),index=False)
    


In [147]:
def sum_df_emp(v4,v3,col_to_index,name):
    
    v4_pivot=v4.pivot_table(index=col_to_index,aggfunc=sum)

    v3_pivot=v3.pivot_table(index=col_to_index,aggfunc=sum)
    
    df_v4_v3=v4_pivot.merge(v3_pivot,left_index=True,right_index=True,how='outer',suffixes=('_v4','_v3')).fillna(0)

    df_v4_v3['total_emp_kayim_delta_pre']=df_v4_v3['total_emp_2020_v4']/df_v4_v3['total_emp_2020_v3']

    #df_v4_v3['total_emp_2040_delta_pre']=df_v4_v3['total_emp_2040_v4']/df_v4_v3['total_emp_2040_v3']

    #df_v4_v3['add_total_emp_2040_v4']=df_v4_v3['total_emp_2040_v4']-df_v4_v3['total_emp_2020_v4']

    df_v4_v3['add_total_emp_2040_v3']=df_v4_v3['total_emp_2040_v3']-df_v4_v3['total_emp_2020_v3']

    #df_v4_v3['add_total_emp_2040_delta_pre']=df_v4_v3['add_total_emp_2040_v4']/df_v4_v3['add_total_emp_2040_v3']

    

    col=['total_emp_2020_v3', 'total_emp_2020_v4','total_emp_kayim_delta_pre']

    df_v4_v3[col].reset_index().to_excel('{}\{}_emp_kayim_comparison.xlsx'.format(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\output',name),index=False)

    col=['total_emp_2040_v3']#,'total_emp_2040_v4','total_emp_2040_delta_pre']

    df_v4_v3[col].reset_index().to_excel('{}\{}_emp_2040_comparison.xlsx'.format(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\output',name),index=False)

    col=['add_total_emp_2040_v3']#,'add_total_emp_2040_v4','add_total_emp_2040_delta_pre']

    df_v4_v3[col].reset_index().to_excel('{}\{}_emp_add_comparison.xlsx'.format(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\output',name),index=False)
    


## קוד

### העלת מידע

העלת גירסא קודמת

In [148]:
file_path=r'W:\Model Versions\3.1\Forecasts'
file_name='220209_תחזיות_V3.1_Published_for_outside'

v3=up_load_df(file_path,file_name)

העלת גירסא קיימת

In [149]:
file_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\join_pop'
file_name='230710_pop_2050_jtmt'

v4=up_load_df(file_path,file_name)

file_path=r'\\FILE-SRV\Jtmt\projections_team\כללי\פעולות\יצירת_מצב_קיים_למס\output'
file_name='2020_jtmt_forcast_full_230710'

v4_kayim=up_load_df(file_path,file_name)

In [150]:
v4_kayim['total_emp_2020']=v4_kayim[[ 'emp_Education', 'emp_okev', 'emp_not_okev']].sum(axis=1)

In [151]:
col_v4=['Taz_num',
 'aprt',
 'pop']

col_v4_kayim=['Taz_num','Taz_name',
 'Muni_Heb','zonetype','main_secto','aprt_20','pop_without_dorms_yeshiva','total_emp_2020']

In [152]:
v4=pd.merge(v4_kayim[col_v4_kayim],v4[col_v4],on='Taz_num',how='left')

### שינוי שדות ותוכן גירסא ישנה

In [153]:
file_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\input'
file_name='Convert columns between versions'

col_convert=up_load_df(file_path,file_name)

v3=v3[list(col_convert.v3)]

v3.columns=list(col_convert.v4)

המרה של טקטס תוכן העמודות גירסא 3

In [154]:
file_path=r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\jtmt\Comparison_to_previous_versions\input'
file_name='machoz'

v3=rename_text_in_col(v3,file_path,file_name,'v3','v4','zonetype')


file_name='sector'

v3=rename_text_in_col(v3,file_path,file_name,'v3','v4','main_secto')


file_name='muni'

v3=rename_text_in_col(v3,file_path,file_name,'v3','v4','Muni_Heb')

### יצירת טבלאות סיכום לחיבור

In [155]:
sum_df_emp(v4,v3,['zonetype','Muni_Heb','main_secto'],'zone_muni_main_secto')

In [132]:
sum_df_pop(v4,v3,['Muni_Heb'],'Muni_Heb')